# CS 109A/AC 209A/STAT 121A Data Science: Project Milestone 4/5
**Harvard University**<br>
**Fall 2016**<br>

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.ensemble import AdaBoostClassifier as AdaBoost
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import grid_search
from sklearn.decomposition import PCA
from sklearn import feature_selection as fs
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Step 1: Explore and Clean Data

First task will be to read, explore, and clean the data. Our first version of the data exploration can be found [here](ExploreData.ipynb). There was a lot of exploratory and debugging code, so we branched off ot this notebook with the findings we thought would be appropriate for this milestone of the project.

In [2]:
#Load and inspect the ncds data
ncds_data = pd.read_csv('datasets/ncds0123.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_data.shape
ncds_data.head()

Shape of data: (18558, 1765)


,ncdsid,n622,n0region,n1region,n2region,n3region,n553,n545,n520,n490,...,n1849,dvht07,dvht11,dvht16,dvrwt07,dvrwt11,dvrwt16,dvwt07,dvwt11,dvwt16
0,N10001N,2,9,9,9,9,23,4,2,12,...,-1,1.21899986267032,1.47299957275415,1.59999942779607,110.347991943347,98.1929931640604,105.055999755876,25.8549957275385,37.6489868164152,56.0199890136717
1,N10002P,1,9,8,8,8,34,4,5,1,...,-1,1.34599971771224,-1,-1,90.865997314449,-1,-1,26.3089904785155,-1,-1
2,N10003Q,1,4,4,4,4,34,4,10,1,...,-1,1.32099914550831,1.49899959564243,1.87999916076665,87.9599914550983,96.4049987792867,89.382995605487,24.4939880371087,38.1019897460905,66.6799926757659
3,N10004R,2,1,1,1,1,26,4,11,1,...,-1,1.29499912262003,1.51099967956562,1.62999916076665,105.16198730471,111.588989257796,132.054992675766,28.122985839843,45.8139953613169,72.7999877929584
4,N10005S,2,10,10,10,10,25,4,1,3,...,-1,1.34599971771224,-1,-1,129.382995605487,-1,-1,37.6489868164152,-1,-1


In [3]:
#Load and inspect the pms additions data
ncds_pms_data = pd.read_csv('datasets/ncds_pms_additionals.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_pms_data.shape
ncds_pms_data.head()

Shape of data: (16990, 54)


,NCDSID,N622,BSTATUS,POD,BOOKING,PLANC,DIASTOL,MAXDBP,ALBECL,XRAY,...,DTB8,DTB9,DTB10,ILLNESS,MOD,TOD,AAD,SBNND,PLCWGT,TABLE62
0,N10001N,2,0,8,8,2,1,1,0,0,...,0,0,0,0,0,-1,-1,-1,-2,-1
1,N10002P,1,0,2,0,4,4,3,0,0,...,0,0,0,0,0,-1,-1,-1,-2,-1
2,N10003Q,1,0,8,8,2,1,3,0,0,...,0,0,1,3,0,-1,-1,-1,-2,-1
3,N10004R,2,0,8,8,2,1,-8,-8,1,...,1,0,0,-1,0,-1,-1,-1,-2,-1
4,N10005S,2,0,8,8,2,1,3,0,1,...,0,0,0,0,0,-1,-1,-1,-2,-1


In [4]:
#Load and inspect the response data
ncds_response_data = pd.read_csv('datasets/ncds_response.txt', delimiter='\t', low_memory=False)
# Print shapes
print "Shape of data:", ncds_response_data.shape
ncds_response_data.head()

Shape of data: (18558, 18)


,NCDSID,N622,BSTATUS,COBIRTH,MULTIPNO,MULTCODE,ETHNICID,OUTCME00,OUTCME01,OUTCME02,OUTCME03,OUTCME04,OUTCME05,OUTCME06,OUTCMEBM,OUTCME07,OUTCME08,OUTCME09
0,N10001N,2,0,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
1,N10002P,1,0,1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1
2,N10003Q,1,0,1,-1,-1,1,1,1,1,1,7,7,7,6,7,7,7
3,N10004R,2,0,1,-1,-1,1,1,1,1,1,1,1,1,1,2,2,3
4,N10005S,2,0,2,-1,-1,5,1,1,1,1,2,2,2,6,6,6,6


#### Step 1.1: Understand the data:

1. Explore the data
2. Understand the predictors, what they mean in real life
3. Understand the values of each predictors
4. Join appropriate datasets

In [6]:
# Columns the help us identify if the patient has epilepsy
epil_columns = ["n390","n391","n392","n415", "n1842", "n1307", "n1308", "n1309", "n1314", "n1317", "n1477", "n1478", "n1479", "n2416", "n2663", "n2664", "n2665"
                , "n2666", "n2667", "n1893", "n1894", "n1895", "n1904", "n1910"]

def evaluate_data(df):
    # Check for range of unique values for the train data
    for i in range(df.shape[1]):
        vals = np.unique(df.iloc[:, i])
        if len(vals) < 15:
            print '(Categorical) {} unique values - {}: {}'.format(len(vals), df.columns[i], vals)
        else:
            print '(Continuous) range of values - ', df.columns[i], ': {} to {}'.format(df.iloc[:, i].min(), df.iloc[:, i].max())

def evaluate_epil_columns(df):
    for column in epil_columns:
        vals = np.unique(df[column])
        if len(vals) < 15:
            print '(Categorical) {} unique values - {}: {}'.format(len(vals), column, vals)
        else:
            print '(Continuous) range of values - ', column, ': {} to {}'.format(df[column].min(), df[column].max())


def columns_with_null(df):
    for column in df.columns:
        df_missing = df[df[column].isnull()]
        count = 0
        if df_missing.shape[0] > 0:
            print "Predictor " , column, " contain null values / Count = " ,df_missing.shape[0]
            count = count +1
    print "Total number of columns with null:",count

In [8]:
# Join datasets
ncds_merged_data = pd.merge(left=ncds_data,right=ncds_pms_data,how='left',left_on='ncdsid',right_on='NCDSID')
ncds_merged_data = pd.merge(left=ncds_merged_data,right=ncds_response_data,how='left',left_on='ncdsid',right_on='NCDSID')
print "Shape of data:", ncds_merged_data.shape

Shape of data: (18558, 1837)


In [32]:
# Evalute the ncds data
#evaluate_data(ncds_merged_data)

#### Step 1.2: Handle missing data:

Are there any missing values, if there are:
1. Can we impute them based on some algorithm
2. Remove or ignore them
3. Assume values based on common sense or prior knowledge

In [24]:
# Remove spaces from data
def convert_spaces_to_null(data):
    data = data.replace([' '],[None]) 
    return data

def fill_with_median(x_fill):
    x_fill = x_fill.groupby(x_fill.columns, axis = 1).transform(lambda x: x.fillna(x.median()))
    return x_fill

def fill_with_mean(x_fill):
    x_fill = x_fill.groupby(x_fill.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))
    return x_fill

def impute_pms_columns(x_fill):
    for index, row in x_fill.iterrows():
#         # 0-3D Sex of child 
#         if pd.isnull(row["N622"]):
#             x_fill.set_value(index, 'N622', -1.0)
        # Reconciled Birth Status  
#         if pd.isnull(row["BSTATUS"]):
#             x_fill.set_value(index, 'BSTATUS', 0)
        # Q6:Place of Delivery  
        if pd.isnull(row["POD"]):
            x_fill.set_value(index, 'POD', 5.0)
        # Q26b: Booking In place  
        if pd.isnull(row["BOOKING"]):
            x_fill.set_value(index, 'BOOKING', 3.0)
        # Q21b: Place of Antenatal care  
        if pd.isnull(row["PLANC"]):
            x_fill.set_value(index, 'PLANC', -8.0)
        # Q29a: Diastolic Blood Pressure  
        if pd.isnull(row["DIASTOL"]):
            x_fill.set_value(index, 'DIASTOL', -2.0)
        # Q29b: Maximum Diatolic Blood Pressure 
        if pd.isnull(row["MAXDBP"]):
            x_fill.set_value(index, 'MAXDBP', -2.0)
        # Q31: Albuminuria and Eclampsia
        if pd.isnull(row["ALBECL"]):
            x_fill.set_value(index, 'ALBECL', -8.0)
        # Q36: X-Ray given
        if pd.isnull(row["XRAY"]):
            x_fill.set_value(index, 'XRAY', -8.0)
        # Q37: Obstetric, pregnancy abnormality - No information
        if pd.isnull(row["ABNORM0X"]):
            x_fill.set_value(index, 'ABNORM0X', -2.0)
        # Q37: No Obstetric, pregnancy abnormality
        if pd.isnull(row["ABNORM00"]):
            x_fill.set_value(index, 'ABNORM00', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Diabetes
        if pd.isnull(row["ABNORM01"]):
            x_fill.set_value(index, 'ABNORM01', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Heart 
        if pd.isnull(row["ABNORM02"]):
            x_fill.set_value(index, 'ABNORM02', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Active TB 
        if pd.isnull(row["ABNORM03"]):
            x_fill.set_value(index, 'ABNORM03', -2.0)
        # Q37: Obstetric, pregnancy abnormality - influenza 
        if pd.isnull(row["ABNORM04"]):
            x_fill.set_value(index, 'ABNORM04', -2.0)
        # Q37: Obstetric, pregnancy abnormality - German Measles 
        if pd.isnull(row["ABNORM05"]):
            x_fill.set_value(index, 'ABNORM05', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Disproportion 
        if pd.isnull(row["ABNORM06"]):
            x_fill.set_value(index, 'ABNORM06', -2.0)
        # Q37: Obstetric, pregnancy abnormality - External version 
        if pd.isnull(row["ABNORM07"]):
            x_fill.set_value(index, 'ABNORM07', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Epilepsy 
        if pd.isnull(row["ABNORM08"]):
            x_fill.set_value(index, 'ABNORM08', -2.0)
        # Q37: Obstetric, pregnancy abnormality - Other 
        if pd.isnull(row["ABNORM09"]):
            x_fill.set_value(index, 'ABNORM09', -2.0)
        # Q37: Bleeding in Pregnancy and before delivery 
        if pd.isnull(row["BLEED"]):
            x_fill.set_value(index, 'BLEED', -1.0)
        # Q38a: Admission to hospital  
        if pd.isnull(row["AD2HOSP"]):
            x_fill.set_value(index, 'AD2HOSP', -1.0)
        # Q39: Type of Labour or Delivery Admission (Hospital)  
        if pd.isnull(row["ADTYPE"]):
            x_fill.set_value(index, 'ADTYPE', -1.0)
        # Q44: Presenting Part  
        if pd.isnull(row["PRESENT"]):
            x_fill.set_value(index, 'PRESENT', -1.0)
        # Q49a: No drugs of this type  
        if pd.isnull(row["LDRUG00"]):
            x_fill.set_value(index, 'LDRUG00', -2.0)
        # Q49a: Chloral, Welldorm  
        if pd.isnull(row["LDRUG01"]):
            x_fill.set_value(index, 'LDRUG01', -2.0)
        # Q49a: Barbiturate   
        if pd.isnull(row["LDRUG02"]):
            x_fill.set_value(index, 'LDRUG02', -2.0)
        # Q49a: Heroin   
        if pd.isnull(row["LDRUG03"]):
            x_fill.set_value(index, 'LDRUG03', -2.0)
        # Q49a: Largactil (chlorpomazine)   
        if pd.isnull(row["LDRUG04"]):
            x_fill.set_value(index, 'LDRUG04', -2.0)
        # Q49a: Sparine (promazine)    
        if pd.isnull(row["LDRUG05"]):
            x_fill.set_value(index, 'LDRUG05', -2.0)
        # Q49a: Phenergan (promethazine)    
        if pd.isnull(row["LDRUG06"]):
            x_fill.set_value(index, 'LDRUG06', -2.0)
        # Q49a: Doriden    
        if pd.isnull(row["LDRUG07"]):
            x_fill.set_value(index, 'LDRUG07', -2.0)
        # Q49a: Oblivon    
        if pd.isnull(row["LDRUG08"]):
            x_fill.set_value(index, 'LDRUG08', -2.0)
        # Q49a: Other    
        if pd.isnull(row["LDRUG09"]):
            x_fill.set_value(index, 'LDRUG09', -2.0)
        # Q50: Anaesthetic    
        if pd.isnull(row["ATHETIC"]):
            x_fill.set_value(index, 'ATHETIC', -2.0)
        # Q55: Resuscitation    
        if pd.isnull(row["RESUS"]):
            x_fill.set_value(index, 'RESUS', -2.0)
        # Q56: Drugs to baby (None)      
        if pd.isnull(row["DTB1"]):
            x_fill.set_value(index, 'DTB1', -2.0)
        # Q56: Drugs to baby (Coranine)       
        if pd.isnull(row["DTB2"]):
            x_fill.set_value(index, 'DTB2', -2.0)
        # Q56: Drugs to baby (Lobeline)       
        if pd.isnull(row["DTB3"]):
            x_fill.set_value(index, 'DTB3', -2.0)
        # Q56: Drugs to baby (Sedatives)       
        if pd.isnull(row["DTB4"]):
            x_fill.set_value(index, 'DTB4', -2.0)
        # Q56: Drugs to baby (Antagonists, nalorphine, levalorfan)       
        if pd.isnull(row["DTB5"]):
            x_fill.set_value(index, 'DTB5', -2.0)
        # Q56: Drugs to baby (Synkavit, Vikastab)      
        if pd.isnull(row["DTB6"]):
            x_fill.set_value(index, 'DTB6', -2.0)
        # Q56: Drugs to baby (Sulphonamides)      
        if pd.isnull(row["DTB7"]):
            x_fill.set_value(index, 'DTB7', -2.0)
        # Q56: Drugs to baby (Penicilin)      
        if pd.isnull(row["DTB8"]):
            x_fill.set_value(index, 'DTB8', -2.0)
        # Q56: Drugs to baby (Streptomycin)       
        if pd.isnull(row["DTB9"]):
            x_fill.set_value(index, 'DTB9', -2.0)
        # Q56: Drugs to baby (Other antibiotics)       
        if pd.isnull(row["DTB10"]):
            x_fill.set_value(index, 'DTB10', -2.0)
        # Q59: Baby's Illness       
        if pd.isnull(row["ILLNESS"]):
            x_fill.set_value(index, 'ILLNESS', -1.0)
        # Q61: Month of Death       
        if pd.isnull(row["MOD"]):
            x_fill.set_value(index, 'MOD', 0.0)
        # Q61: Time of death        
        if pd.isnull(row["TOD"]):
            x_fill.set_value(index, 'TOD', -1.0)
        # Q61: Age at Death        
        if pd.isnull(row["AAD"]):
            x_fill.set_value(index, 'AAD', -1.0)
        # Q61: Still Birth or Neo-natal Death (Dervied)         
        if pd.isnull(row["SBNND"]):
            x_fill.set_value(index, 'SBNND', -1.0)
        # Placental Weight         
        if pd.isnull(row["PLCWGT"]):
            x_fill.set_value(index, 'PLCWGT', -2.0)
        # Time of death for still births and neonatal deaths (Table 62)          
        if pd.isnull(row["TABLE62"]):
            x_fill.set_value(index, 'TABLE62', -1.0)
    return x_fill

In [25]:
# Make a copy
ncds_data_clean = ncds_merged_data.copy() 

# drop the ID columns
ncds_data_clean =  ncds_data_clean.drop(["ncdsid","NCDSID_x","NCDSID_y"],axis=1)
# Drop other duplicate columns
ncds_data_clean =  ncds_data_clean.drop(["N622_x","BSTATUS_x"],axis=1)


# Convert spaces in the data to nulls
ncds_data_clean = convert_spaces_to_null(ncds_data_clean)

# Convert all columns to float
for column in ncds_data_clean.columns:
    ncds_data_clean[column] = ncds_data_clean[column].astype(float)

# Impute missing data from joined columns using default values
ncds_data_clean = impute_pms_columns(ncds_data_clean)

# Impute missing data with median values
ncds_data_clean = fill_with_median(ncds_data_clean)

# Impute missing data with mean values - there are some columns we cannot impute with median
ncds_data_clean = fill_with_mean(ncds_data_clean)


In [26]:
# Get the columns which have null data
columns_with_null(ncds_data_clean)

Total number of columns with null: 0


#### Step 1.3: Identify Epilepsy Records:

In our data a patient is assumed to be epileptic is one or more conditions are satisified in the dataset. We need to check all the conditions in the data and determine if the patient is epileptic.

In [27]:
# Identify if patient has epilepsy
ncds_data_clean["epileptic"] = 0
for index, row in ncds_data_clean.iterrows():
    # 1M Reason for Special Education MC1:3
    if row["n390"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Reason for Special Education MC2:3
    if row["n391"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Reason for Special Education MC2:3
    if row["n392"] == 10.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 1M Epileptic condition
    if row["n415"] >= 3.0 :
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 12D Epilepsy identification
    if row["n1842"] == 5.0 :
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 1:3
    if (row["n1307"] >= 1.0 and row["n1307"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 2:3
    if (row["n1308"] >= 1.0 and row["n1308"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Has child had epilepsy attacks-MC 3:3
    if (row["n1309"] >= 1.0 and row["n1309"] <= 5.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Age at most recent epilepsy attack
    if (row["n1314"] >= 0.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2P Age at 1st epilepsy attack
    if (row["n1317"] >= 0.0):
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC1:3
    if row["n1477"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC2:3
    if row["n1478"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 2M Reason for special education - MC3:3
    if row["n1479"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Type hcap for which will require help
    if row["n2416"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 1:5
    if row["n2663"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 2:5
    if row["n2664"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 3:5
    if row["n2665"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 4:5
    if row["n2666"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3P Nature of child-s disability-MC 5:5
    if row["n2667"] == 7.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC1:3
    if row["n1893"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC2:3
    if row["n1894"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Category of child's handicap MC3:3
    if row["n1895"] == 8.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Reason for hosp admiss last 12 mnths
    if row["n1904"] == 17.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Reason hosp outpatient last yr
    if row["n1910"] == 17.0:
        ncds_data_clean.set_value(index, 'epileptic', 1)
    # 3M Epilepsy
#     if row["n2032"] >= 1.0:
#         ncds_data_clean.set_value(index, 'epileptic', 1)

In [28]:
print "Number of rows with epilepsy",ncds_data_clean[ncds_data_clean["epileptic"] == 1].shape[0]

Number of rows with epilepsy 811


In [29]:
# Remove the epilepsy columns from the data
ncds_data_no_indicators=ncds_data_clean.copy()
ncds_data_no_indicators.drop(epil_columns,inplace=True,axis=1)
print "Shape of dataset: " , ncds_data_no_indicators.shape

Shape of dataset:  (18558, 1809)


#### Step 1.4: Split data into train and test:

Split our dataset into train and test and analyze the splits. We can explore and verify the matrix of classes to check if our data is balanced. If the class is Imbalanced we will need to do any of the following:
1. Over sample
2. Under sample
3. Over weight
4. Adjust class weights in model

In [31]:
x = ncds_data_no_indicators.values[:, :-1]
y = ncds_data_no_indicators.values[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

#Print some useful info for our test, train sets
print 'Train data: ', x_train.shape
print 'Test data: ', x_test.shape
print 'Train class 0: {}, train class 1: {}'.format(len(y_train[y_train == 0]), len(y_train[y_train == 1]))
print 'Test class 0: {}, test class 1: {}'.format(len(y_test[y_test == 0]), len(y_test[y_test == 1]))

Train data:  (11134, 1808)
Test data:  (7424, 1808)
Train class 0: 10647, train class 1: 487
Test class 0: 7100, test class 1: 324


## Step 2: Feature Selection

From the merged datasets we can see we have over 1800 features. Going through the 1800 would be a very time consuming task so let us apply some algorithims to find the best features that we can use to build the model

## Step 3: Build Base Models

In [30]:
# Function for computing the accuracy a given model on the entire test set,
# the accuracy on class 0 in the test set
# and the accuracy on class 1
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])],
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])


#### Step 3.1: Linear Regression:

#### Step 3.2: Logistic Regression:

#### Step 3.1: Linear Discriminant Analysis:

#### Step 3.1: Other Models: